In [1]:
from tensorflow.keras.layers import Dense, Input, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
batch_size = 64
width, height, channel = 400, 400, 3
no_epochs = 10
verbosity = 2
input_shape = (width, height, channel)

In [3]:
# ResNet50 모델 사용

pretrained_model = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=1000
)

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('preprocessed_data/train',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('preprocessed_data/test',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

# pretrained_model.trainable=True
# set_trainable=False

# for layer in pretrained_model.layers:
#     if layer.name == 'block7c_project_conv':
#         set_trainable=True
#     if set_trainable:
#         layer.trainable=True
#     else:
#         layer.trainable=False


model=Sequential()
    
# model.add(pretrained_model)

for layer in pretrained_model.layers[:-1]: # just exclude last layer from copying
    model.add(layer)
    
for layer in model.layers:
    layer.trainable = False

model.add(tf.keras.layers.Conv2D(filters=1024, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(tf.keras.layers.GlobalAveragePooling2D())
# model.add(tf.keras.layers.Dense(1, activation='softmax'))

# model.add(MaxPooling2D())
# model.add(Dropout(0.2))
# model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.AUC(), 'accuracy'])

Found 2603 images belonging to 2 classes.
Found 650 images belonging to 2 classes.


In [6]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 400, 400, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 400, 400, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 200, 200, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 200, 200, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 200, 200, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 100, 100, 128)     0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 100, 100, 256)    

In [ ]:
es_callback = EarlyStopping(monitor='val_auc', mode='max', patience=8,
                                              verbose=1, min_delta=0.0001, restore_best_weights=True)

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//batch_size,
                    epochs = no_epochs,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//batch_size,
                    callbacks= [es_callback],
                    verbose=verbosity,
                   )

Epoch 1/10
40/40 - 31s - loss: 0.7190 - auc: 0.6357 - accuracy: 0.5825 - val_loss: 0.5540 - val_auc: 0.8529 - val_accuracy: 0.7891
Epoch 2/10
40/40 - 28s - loss: 0.5231 - auc: 0.8274 - accuracy: 0.7495 - val_loss: 0.4680 - val_auc: 0.8859 - val_accuracy: 0.8203
Epoch 3/10
40/40 - 28s - loss: 0.4471 - auc: 0.8826 - accuracy: 0.8109 - val_loss: 0.4011 - val_auc: 0.9158 - val_accuracy: 0.8531
Epoch 4/10
40/40 - 29s - loss: 0.4037 - auc: 0.9026 - accuracy: 0.8267 - val_loss: 0.3592 - val_auc: 0.9336 - val_accuracy: 0.8578
Epoch 5/10
40/40 - 29s - loss: 0.3660 - auc: 0.9180 - accuracy: 0.8432 - val_loss: 0.3331 - val_auc: 0.9438 - val_accuracy: 0.8594
Epoch 6/10
40/40 - 29s - loss: 0.3555 - auc: 0.9200 - accuracy: 0.8444 - val_loss: 0.3009 - val_auc: 0.9509 - val_accuracy: 0.8875
Epoch 7/10
40/40 - 29s - loss: 0.3385 - auc: 0.9268 - accuracy: 0.8456 - val_loss: 0.2920 - val_auc: 0.9533 - val_accuracy: 0.8828
Epoch 8/10
40/40 - 28s - loss: 0.2946 - auc: 0.9483 - accuracy: 0.8748 - val_loss: 

In [ ]:
prediction = model.predict(valid_generator)

In [ ]:
import numpy as np

print(len(prediction[np.where(prediction > 0.5)]))
print(len(prediction[np.where(prediction < 0.5)]))

In [ ]:
prediction

In [ ]:
model.evaluate(valid_generator)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from keras.preprocessing import image
img = image.load_img('preprocessed_data/test/positive/positive_0_2404.jpg', target_size=(400, 400, 3))

get_output = tf.keras.backend.function([model.layers[0].input],
                                       [model.layers[-3].output, model.layers[-1].output])
# [conv_outputs, predictions] = get_output([valid_generator[0][0][0][0]])
# class_weights = model.layers[-1].get_weights()[0]

In [ ]:
valid_generator[0][0][0][0]